### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра математического моделирования и искусственного интеллекта

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2

#### Дисциплина: Интеллектуальный анализ данных

##### Студент:  Миронов Дмитрий Андреевич
##### Группа:   НПИбд-02-21

## Москва 2024
***

### Вариант № 6

Алгоритм: FPGrowth

День недели (поле order_dow таблицы orders): “0”

Код департамента (поле department_id таблицы products): “8”  

Запрос: определить список товаров, которые были приобретены ровно два раза

Показатель оценки ассоциативных правил: лифт (lift)

1.	При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.

In [128]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [129]:
import sqlite3
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

In [130]:
conn = sqlite3.connect('instacart (1).db')
cursor = conn.cursor()
tables = cursor.fetchall()
print("Таблицы в базе данных Instacart:", tables)
type(cursor)

Таблицы в базе данных Instacart: []


sqlite3.Cursor

2.	Загрузите таблицы departments и products в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы order_products__train записи, соответствующие указанным в индивидуальном задании дню недели (поле order_dow таблицы orders) и коду департамента (поле department_id таблицы products) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме, количество транзакций (покупок) и определите количество товаров (столбец product_id) в транзакциях датафрейма.

In [131]:
departments = pd.read_sql_query("SELECT * FROM departments", conn)
products = pd.read_sql_query("SELECT * FROM products", conn)

In [132]:
departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [133]:
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [134]:
query = """
SELECT opt.*
FROM order_products__train opt
JOIN orders o ON opt.order_id = o.order_id
JOIN products p ON opt.product_id = p.product_id
WHERE o.order_dow = 0 AND p.department_id = 8
"""

In [114]:
df = pd.read_sql_query(query, conn)
df

,order_id,product_id,add_to_cart_order,reordered
0,2350551,26,4,1
1,2646573,26,15,1
2,1107474,54,18,1
3,1227234,54,13,1
4,1397986,54,2,0
...,...,...,...,...
883,3396864,49587,18,1
884,506992,49587,3,1
885,832450,49587,2,1
886,1731934,49590,6,0


In [115]:
print(f"Количество строк: {len(df)}")
print(f"Количество транзакций: {df['order_id'].nunique()}")
print(f"Количество товаров: {df['product_id'].nunique()}")

Количество строк: 888
Количество транзакций: 551
Количество товаров: 374


3. Выполните к датафрейму запрос, указанный в индивидуальном задании.

In [116]:
query_ = "SELECT product_id FROM order_products__train GROUP BY product_id HAVING COUNT(product_id) = 2; "

In [117]:
df_ = pd.read_sql_query(query_, conn)
df_

,product_id
0,10000
1,1001
2,10026
3,10035
4,10039
...,...
4905,9942
4906,9954
4907,9960
4908,9979


In [118]:
df_ = pd.read_sql_query(query_, conn)

max_product = df_['product_id'].iloc[0]
max_purchases = 2

print(f"Товар с наибольшим количеством покупок (среди купленных дважды): {max_product}")
print(f"Количество покупок этого товара: {max_purchases}")

Товар с наибольшим количеством покупок (среди купленных дважды): 10000
Количество покупок этого товара: 2


4. Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец order_id, а в качестве названий товаров - поле product_name из датафрейма для таблицы products, соответствующее столбцу product_id. Найдите в транзакционной базе данных транзакцию с наибольшим количеством товаров и выведите ее на экран.

In [119]:
df_merged = pd.merge(df, products[['product_id', 'product_name']], on='product_id', how='left')
transaction_db = df_merged.groupby('order_id')['product_name'].apply(list).reset_index()
transaction_db

,order_id,product_name
0,100000,[Snack Sticks Chicken & Rice Recipe Dog Treats]
1,1000247,[Whiskas® Temptations® Mix Ups Backyard Cookou...
2,1005229,"[Classic Tender Beef Feast Cat Food, Fancy Fea..."
3,1006312,[Scented Multi-Cat Scoopable Cat Litter]
4,1011575,"[Dog Snacks, Medium, Standard Size Pet Waste b..."
...,...,...
546,967248,[Buddy Biscuits Original Oven Baked Roasted Ch...
547,968278,[Complete Cat Food]
548,973753,[Proactive Health Indoor Weight & Hairball Car...
549,992780,[Super Scoop Instant Clumping Unscented Cat Li...


5.	Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета mlxtend. По бинарной базе данных определите и выведите на экран три наиболее популярных товара и определите количество покупок (транзакций) этих товаров.

In [120]:
te = TransactionEncoder()
transaction_bin = te.fit(transaction_db['product_name']).transform(transaction_db['product_name'])
binary_df = pd.DataFrame(transaction_bin, columns=te.columns_)
binary_df

,174 ; Dentastix&#174 ; Large,24/7 Performance Cat Litter,24/7 Performance Clumping Litter,24/7 Performance Light Weight Cat Litter,5 Meaty Flavors Variety Dog Snacks,"6\"" Free Ranger Bully Stix",Adult Chicken & Rice Dry Dog Food,Adult Dog Food,Adult Dog Formula Dog Food,Adult Grain-Free & Poultry-Free Dog Food,...,White Meat Chicken Cutlets Natural Dog Snacks,White Meat Chicken Florentine Cat Food,White Meat Chicken Primavera Cat Food,Wild Salmon Florentine Cat Food,With Chicken & Veal in Meaty Juices Wet Dog Food,With Glade Tough Odor Solutions Cat Litter,With Ocean Fish & Salmon Cat Food,With Savory Chicken & Turkey Cat Food,Z Filets Grilled Beef Dog Treats,smart Blend Chicken & Rice Formula Dry Dog Food
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
547,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
548,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
549,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [121]:
counts = binary_df.sum().sort_values(ascending=False)
counts.head(3)

,0
Instant Action Cat Litter,23
24/7 Performance Cat Litter,18
Grain Free Chicken Formula Cat Food,17


6. При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте и выведите на экран популярный набор предметов с (абсолютной) минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии такого набора уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярного набора предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).

In [122]:
min_support = 3 / len(binary_df)
itemsets_FPGrowth = fpgrowth(binary_df, min_support=min_support, use_colnames=True)
itemsets_FPGrowth

,support,itemsets
0,0.016334,(Snack Sticks Chicken & Rice Recipe Dog Treats)
1,0.016334,(Classic Tender Beef Feast Cat Food)
2,0.007260,(Fancy Feast Wet Classic Turkey & Giblets Feas...
3,0.007260,(Scented Multi-Cat Scoopable Cat Litter)
4,0.009074,"(Dog Snacks, Medium)"
...,...,...
116,0.005445,"(Natural Beef & Chicken Formula Food for Cats,..."
117,0.005445,(Fancy Feast Wet Classic Chicken Feast Cat Foo...
118,0.005445,"(Chunky Chicken Feast Cat Food, Classic Tender..."
119,0.005445,(Classic Ocean Whitefish & Tuna Feast Cat Food...


In [123]:
itemsets_FPGrowth['length'] = itemsets_FPGrowth['itemsets'].apply(lambda x: len(x))
itemsets_FPGrowth

,support,itemsets,length
0,0.016334,(Snack Sticks Chicken & Rice Recipe Dog Treats),1
1,0.016334,(Classic Tender Beef Feast Cat Food),1
2,0.007260,(Fancy Feast Wet Classic Turkey & Giblets Feas...,1
3,0.007260,(Scented Multi-Cat Scoopable Cat Litter),1
4,0.009074,"(Dog Snacks, Medium)",1
...,...,...,...
116,0.005445,"(Natural Beef & Chicken Formula Food for Cats,...",2
117,0.005445,(Fancy Feast Wet Classic Chicken Feast Cat Foo...,2
118,0.005445,"(Chunky Chicken Feast Cat Food, Classic Tender...",2
119,0.005445,(Classic Ocean Whitefish & Tuna Feast Cat Food...,2


Популярные наборы товаров с максимальной длиной и поддержкой не менее 3:

In [124]:
max_length = itemsets_FPGrowth[itemsets_FPGrowth['length'] == itemsets_FPGrowth['length'].max()]
max_length

,support,itemsets,length
115,0.005445,"(Grain Free Chicken Formula Cat Food, Grain Fr...",3


In [125]:
rules = association_rules(itemsets_FPGrowth, metric="confidence", min_threshold=0.6)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Fancy Feast Wet Classic Turkey & Giblets Feas...,(Classic Tender Beef Feast Cat Food),0.007260,0.016334,0.005445,0.750000,45.916667,0.005326,3.934664,0.985375
1,(Grilled Turkey Feast in Gravy Cat Food),(Grilled Chicken Feast in Gravy Cat Food),0.009074,0.010889,0.007260,0.800000,73.466667,0.007161,4.945554,0.995421
2,(Grilled Chicken Feast in Gravy Cat Food),(Grilled Turkey Feast in Gravy Cat Food),0.010889,0.009074,0.007260,0.666667,73.466667,0.007161,2.972777,0.997248
3,(Grilled Turkey Feast in Gravy Cat Food),(Fancy Feast Wet Classic Chicken Feast Cat Food),0.009074,0.018149,0.005445,0.600000,33.060000,0.005280,2.454628,0.978632
4,(Grain Free Turkey Formula Cat Food),(Grain Free Turkey & Salmon Formula Cat Food),0.014519,0.021779,0.009074,0.625000,28.697917,0.008758,2.608590,0.979374
5,"(Grain Free Chicken Formula Cat Food, Grain Fr...",(Grain Free Turkey Formula Cat Food),0.009074,0.014519,0.005445,0.600000,41.325000,0.005313,2.463702,0.984737
6,"(Grain Free Chicken Formula Cat Food, Grain Fr...",(Grain Free Turkey & Salmon Formula Cat Food),0.007260,0.021779,0.005445,0.750000,34.437500,0.005287,3.912886,0.978062
7,"(Grain Free Turkey & Salmon Formula Cat Food, ...",(Grain Free Chicken Formula Cat Food),0.009074,0.030853,0.005445,0.600000,19.447059,0.005165,2.422868,0.957265
8,(Natural Beef & Chicken Formula Food for Cats),(Grain Free Turkey Formula Cat Food),0.007260,0.014519,0.005445,0.750000,51.656250,0.005339,3.941924,0.987812
9,(Classic Ocean Whitefish & Tuna Feast Cat Food),(Classic Tender Liver & Chicken Feast Cat Food),0.007260,0.012704,0.005445,0.750000,59.035714,0.005352,3.949183,0.990250


8.	Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и выведите на экран ассоциативное правило, имеющее наилучшее значение этого показателя оценки.

In [126]:
rules.loc[rules['lift'].idxmax()]

,10
antecedents,(Chunky Beef with Vegetables & Brown Rice Dog ...
consequents,(Select Tender Chicken with Vegetables & Brown...
antecedent support,0.00726
consequent support,0.00726
support,0.005445
confidence,0.75
lift,103.3125
leverage,0.005392
conviction,3.970962
zhangs_metric,0.997562


In [127]:
rules.sort_values(by='leverage', ascending=False).head(2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4,(Grain Free Turkey Formula Cat Food),(Grain Free Turkey & Salmon Formula Cat Food),0.014519,0.021779,0.009074,0.625,28.697917,0.008758,2.608590,0.979374
1,(Grilled Turkey Feast in Gravy Cat Food),(Grilled Chicken Feast in Gravy Cat Food),0.009074,0.010889,0.007260,0.800,73.466667,0.007161,4.945554,0.995421
